In [31]:
!pip install pymysql
!pip install apyori
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install python-dotenv

In [1]:
# Imported Libraries
import pandas as pd
import pymysql

# Data visualisation tools for later.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori

# Load the .env file for database credentials
from dotenv import load_dotenv

load_dotenv()

####### Connection to Client Database #######
conn = pymysql.connect(host=os.getenv("host"), user=os.getenv("user"), port=int(os.getenv("port")), password=os.getenv("password"), db=os.getenv("dbname"))

print(conn)

### Getting Customer-users detail - make dataframe

In [8]:
####Checking queries

###Customer Personal details
##  add the desired rows in the query from the three tables
#df_snow_cus = pd.read_sql("select m.id as member_id, m.membership_number, m.status, m.price_zone_code, s.u_ndis_number, s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member m  on s.u_ndis_number = m.membership_number left join libe_leapinprod_memberregion r on r.MemberId =  s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null;", con=conn)
#print(df_snow_cus.info())
#print(df_snow_cus)
df_customer_details = pd.read_sql("select m.id as member_id, m.membership_number, m.status, m.price_zone_code, s.u_ndis_number, s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member m  on s.u_ndis_number = m.membership_number left join libe_leapinprod_memberregion r on r.MemberId =  s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null and s.u_ndis_number != ''", con=conn)

df_customer_details.info()

# df_customer_details["u_disabilities"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   member_id          6115 non-null   float64
 1   membership_number  6115 non-null   object 
 2   status             6115 non-null   object 
 3   price_zone_code    6115 non-null   object 
 4   u_ndis_number      6241 non-null   object 
 5   u_disabilities     6241 non-null   object 
 6   u_gender           6241 non-null   object 
 7   u_date_of_birth    6239 non-null   object 
 8   SA1                6021 non-null   float64
 9   SA2                6021 non-null   float64
 10  SA3                6021 non-null   float64
 11  SA4                6021 non-null   float64
dtypes: float64(5), object(7)
memory usage: 585.2+ KB


### Getting Services Providers detail - make dataframe

In [23]:
####Checking queries
### Service Providers Details:

#! Problem - too few ABNs to meaningfully join with SNOW_customer_account

#df_providers_details = pd.read_sql("select p.id, pa.provider_account_key, p.is_inactive, p.abn, ca.city, ca.country from HH_provider_account pa join HH_provider p on p.id = pa.provider_id join SNOW_customer_account ca on ca.u_abn = p.abn;", con=conn)

#print(df_providers_details.info())

#df_providers_details.head(50)

### Getting Claims and Invoice details - make dataframe

In [19]:
### Checking Queries:
### Getting Claims-Providers details:

##df_invoice_claim_details = pd.read_sql("SELECT c.id as claim_id, c.invoice_id, c.state, c.risk_score, c.risk_level, c.risk_warnings, c.unit_price, c.start_date, c.end_date, c.invoiced_units, c.invoiced_unit_price, c.claimed_units, c.claimed_unit_price, c.claimed_date, c.funded_amount, c.funded_date, c.submitted_date, c.approved_date, c.received_date, c.processed_status, c.reconcile_status, c.ndis_cancelled_date, c.cancel_offset_z4_no, i.state, i.is_pre_approved, i.is_resolved, i.is_risk_identified, i.risk_score, i.invoice_total, i.funded_total, i.funded_date, i.claimed_date, i.reviewed_date, i.invoice_date, i.approved_date, i.submitted_date, i.received_date, m.membership_number, m.status, m.risk_factor, m.price_zone_code FROM HH_claim c left join HH_invoice i on c.invoice_id = i.id left join HH_member m on i.member_id = m.id ;", con=conn)

##df_invoice_claim_details.info()
##df_invoice_claim_details

In [6]:

### Getting Claims details: 

df_claims= pd.read_sql("SELECT c.id as claim_id, c.invoice_id, c.state, c.risk_level, c.start_date FROM HH_claim c;", con=conn)
# df_claims.info()
#df_claims

###Getting Invoice Details

df_invoices= pd.read_sql("SELECT i.id as invoice_id, i.member_id, i.invoice_total, i.funded_total, i.funded_date FROM HH_invoice i;", con=conn)
# df_invoices.info()
#df_invoices

###Getting Member Details
#df_mem= pd.read_sql("SELECT m.id as member_id, m.membership_number, m.status, m.price_zone_code FROM HH_member m;", con=conn)
#df_mem.info()
#df_mem

### Merging
df_claimsWithInvoices = pd.merge(df_claims, df_invoices, on="invoice_id", how="left")
#df_invoice_claim_details = pd.merge(df_claimsWithInvoices, df_mem, on="member_id", how="left")

#df_claimsWithInvoices.info()
df_invoice_claim_details= df_claimsWithInvoices

df_invoice_claim_details.info()

df_invoice_claim_details.head(10)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 968874 entries, 0 to 968873
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   claim_id       968874 non-null  int64  
 1   invoice_id     968874 non-null  int64  
 2   state          968874 non-null  object 
 3   risk_level     968874 non-null  object 
 4   start_date     968874 non-null  object 
 5   member_id      968874 non-null  int64  
 6   invoice_total  968874 non-null  float64
 7   funded_total   566458 non-null  float64
 8   funded_date    556790 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 73.9+ MB


,claim_id,invoice_id,state,risk_level,start_date,member_id,invoice_total,funded_total,funded_date
0,1,1,PAID,NO,2020-04-30,2,160.00,160.00,2020-04-30
1,2,2,PAID,NO,2020-05-01,2,160.00,160.00,2020-05-05
2,3,3,PAID,NO,2020-04-23,3,80.00,80.00,2020-05-08
3,4,4,PAID,NO,2020-04-30,3,80.00,80.00,2020-05-08
4,5,5,PAID,NO,2020-05-07,3,80.00,80.00,2020-05-08
5,6,6,PAID,NO,2020-05-07,4,214.41,214.41,2020-05-12
6,7,7,PAID,NO,2020-05-12,2,130.00,130.00,2020-05-13
7,8,8,PAID,NO,2020-05-14,2,130.00,130.00,2020-05-15
8,9,9,PAID,NO,2020-05-14,4,428.82,428.82,2020-05-18
9,10,10,PAID,NO,2020-05-16,4,214.41,214.41,2020-05-18


### Checking Claims_id and Providers link - making dataframe

In [56]:
### All Claim details (took 2 minutes to execute)
##  
df_all_claims = pd.read_sql("select c.id as claim_id, invoice_id as invoiceId, c.item_category_level3_id from HH_claim c", con=conn)

#print(df_all_claims.info())
#print(df_all_claims)
### All item_categories details. Took few seconds for execution.

##  
df_all_ndis_service_cat = pd.read_sql("select item_category_level3_id, ndis.registration_group from hedgehog_ndis_service_item_ref ndis", con=conn)
#print(df_all_ndis_service_cat.info())
#print(df_all_ndis_service_cat)

##Combine df_all_claims and df_all_ndis_service_cat
df_claimsWithProvider_details = pd.merge(df_all_claims, df_all_ndis_service_cat, on="item_category_level3_id", how= 'inner')
df_claimsWithProvider_details= df_claimsWithProvider_details.drop(columns=['item_category_level3_id'])
#df_claimsWithProvider_details = pd.read_sql("select c.id, c.invoice_id as invoiceId, n.registration_group from HH_claim c left join hedgehog_ndis_service_item_ref n on c.item_category_level3_id = n.item_category_level3_id;", con=conn)
#print(df_claimsWithProvider_details.info())
#print(df_claimsWithProvider_details)

### Provider Services details (Provider account linked to invoice and HH_provider) took 1 minute to execute
##  
df_providersWithInvoice_details = pd.read_sql("select i.id as invoiceId, i.provider_account_id, (select p.abn from HH_provider p where pa.provider_id= p.id) as provider_abn from HH_invoice i left join HH_provider_account pa on i.provider_account_id = pa.id;", con=conn)
#print(df_providersWithInvoice_details.info())
#print(df_providersWithInvoice_details)

##Combine df_claim_providers_details and df_providersWithInvoice_details
df_claims_provider_details= pd.merge(df_claimsWithProvider_details, df_providersWithInvoice_details, on="invoiceId", how="left")

##################################################

#Final claim and Provider df
print(df_claims_provider_details.info())
print(df_claims_provider_details)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 969779 entries, 0 to 969778
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   claim_id             969779 non-null  int64 
 1   invoiceId            969779 non-null  int64 
 2   registration_group   969779 non-null  object
 3   provider_account_id  969779 non-null  int64 
 4   provider_abn         940538 non-null  object
dtypes: int64(3), object(2)
memory usage: 44.4+ MB
None
        claim_id  invoiceId                               registration_group  \
0              1          1                             Therapeutic Supports   
1              2          2                             Therapeutic Supports   
2              3          3                             Therapeutic Supports   
3              4          4                             Therapeutic Supports   
4              5          5                             Therapeutic Supports   
... 

## Merging the dataframes

In [61]:
#Merging invoice, claim and member
df_invoice_claim_cus = pd.merge(df_invoice_claim_details, df_customer_details, on="member_id", how="left")

#The membership_number is 902999 because we have considered the members only with 'li_managed' stage. 
#The records for members having the member id and not having the membership number needs to be dropped.
filter1 = df_invoice_claim_cus ["membership_number"].notnull()
df_invoice_claim_cus.where(filter1, inplace = True)
# df_invoice_claim_cus.info()
# df_invoice_claim_cus

#Merging with service providers
df_final = pd.merge(df_invoice_claim_cus, df_claims_provider_details, on="claim_id", how="left")
df_final= df_final.drop(columns=['invoiceId'])
df_final.info()

df_final.head(1000)



# df_final["u_disabilities"].value_counts()
#There are some claims which do not have a provider specific information



<class 'pandas.core.frame.DataFrame'>
Int64Index: 969735 entries, 0 to 969734
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   claim_id             903860 non-null  float64
 1   invoice_id           903860 non-null  float64
 2   state                903860 non-null  object 
 3   risk_level           903860 non-null  object 
 4   start_date           903860 non-null  object 
 5   member_id            903860 non-null  float64
 6   invoice_total        903860 non-null  float64
 7   funded_total         540423 non-null  float64
 8   funded_date          531471 non-null  object 
 9   membership_number    903860 non-null  object 
 10  status               903860 non-null  object 
 11  price_zone_code      903860 non-null  object 
 12  u_ndis_number        903860 non-null  object 
 13  u_disabilities       903860 non-null  object 
 14  u_gender             903860 non-null  object 
 15  u_date_of_birth  

,claim_id,invoice_id,state,risk_level,start_date,member_id,invoice_total,funded_total,funded_date,membership_number,...,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4,registration_group,provider_account_id,provider_abn
0,1.0,1.0,PAID,NO,2020-04-30,2.0,160.0,160.0,2020-04-30,277486509,...,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0,Therapeutic Supports,157.0,13448919450
1,2.0,2.0,PAID,NO,2020-05-01,2.0,160.0,160.0,2020-05-05,277486509,...,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0,Therapeutic Supports,157.0,13448919450
2,3.0,3.0,PAID,NO,2020-04-23,3.0,80.0,80.0,2020-05-08,815939440,...,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0,Therapeutic Supports,1877.0,454964437
3,4.0,4.0,PAID,NO,2020-04-30,3.0,80.0,80.0,2020-05-08,815939440,...,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0,Therapeutic Supports,1877.0,454964437
4,5.0,5.0,PAID,NO,2020-05-07,3.0,80.0,80.0,2020-05-08,815939440,...,,Female,1937-10-25,3130913.0,31309.0,31102.0,311.0,Therapeutic Supports,1877.0,454964437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.0,560.0,PAID,NO,2019-09-02,21.0,190.0,NaN,None,661532612,...,,Female,1996-08-28,1111516.0,11115.0,10602.0,106.0,Therapeutic Supports,1606.0,40627330376
996,997.0,802.0,PAID,NO,2019-05-01,21.0,175.0,NaN,None,661532612,...,,Female,1996-08-28,1111516.0,11115.0,10602.0,106.0,Therapeutic Supports,986.0,29230435046
997,998.0,564.0,PAID,NO,2019-05-30,21.0,63.0,NaN,None,661532612,...,,Female,1996-08-28,1111516.0,11115.0,10602.0,106.0,Therapeutic Supports,1144.0,32600120787
998,999.0,500.0,PAID,NO,2019-09-20,21.0,304.0,NaN,None,661532612,...,,Female,1996-08-28,1111516.0,11115.0,10602.0,106.0,Therapeutic Supports,4069.0,86432469778


In [79]:

# Merge invoice and claim as normal
df_invoice_claim = pd.merge(df_claims, df_invoices, on="invoice_id", how="left")

df_invoice_claim.head(1000)

# Set member_id as the index of the df
df_invoice_claim = df_invoice_claim.set_index(["member_id"])

# Join customer details on member_id
df_invoice_claim_member = df_invoice_claim.join(df_customer_details.set_index(["member_id"]))

df_invoice_claim_member.head(100)




,claim_id,invoice_id,state,risk_level,start_date,invoice_total,funded_total,funded_date,membership_number,status,price_zone_code,u_ndis_number,u_disabilities,u_gender,u_date_of_birth,SA1,SA2,SA3,SA4
member_id,,,,,,,,,,,,,,,,,,,
2,1,1,PAID,NO,2020-04-30,160.00,160.00,2020-04-30,277486509,managed,ACT_NSW_QLD_VIC,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
2,2,2,PAID,NO,2020-05-01,160.00,160.00,2020-05-05,277486509,managed,ACT_NSW_QLD_VIC,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
2,7,7,PAID,NO,2020-05-12,130.00,130.00,2020-05-13,277486509,managed,ACT_NSW_QLD_VIC,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
2,8,8,PAID,NO,2020-05-14,130.00,130.00,2020-05-15,277486509,managed,ACT_NSW_QLD_VIC,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
2,12,12,PAID,NO,2020-05-14,320.00,320.00,2020-05-21,277486509,managed,ACT_NSW_QLD_VIC,277486509,,Male,1958-04-12,3110117.0,31101.0,30404.0,304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,910550,492186,PAID,NO,2021-01-20,214.41,214.41,2021-03-15,305008838,managed,ACT_NSW_QLD_VIC,305008838,,Male,1987-04-22,3129618.0,31296.0,31004.0,310.0
4,910554,492190,PAID,NO,2021-01-30,214.41,214.41,2021-03-15,305008838,managed,ACT_NSW_QLD_VIC,305008838,,Male,1987-04-22,3129618.0,31296.0,31004.0,310.0
4,914011,494072,PAID,NO,2021-03-13,387.98,387.98,2021-03-16,305008838,managed,ACT_NSW_QLD_VIC,305008838,,Male,1987-04-22,3129618.0,31296.0,31004.0,310.0
